In [ ]:
!pip install xmltodict

## 기준이 되는 발생신용손실

In [ ]:
import requests
#xml 을 딕셔너리 구조로 보여주는 xmltodict 라이브러리 사용
import xmltodict
import pprint as pp
import requests
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import json

ECOS_KEY = "17RCR3XU541NV6BJAJJ7"
sales_loss_url= "https://ecos.bok.or.kr/api/StatisticSearch/17RCR3XU541NV6BJAJJ7/json/kr/1/1000/501Y002/A/2018/2020"

def get_json_response(url, service_name="API"):
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"❌ {service_name} 요청 실패: {resp.status_code}")
        return None
    try:
        return resp.json()
    except ValueError:
        print(f"❌ {service_name} 응답이 JSON이 아님")
        return None

# ---------------------------
# 1. 대손상각비 write-off - 단위 백만원
# 2. 부실채권 매각손실 = 투자·유형자산 처분손실   Loss on Disposition of Tangible Assets  - 단위 백만원
# 3. 기타 손실조정 = 기타영업외비용 Other Non-Operating Expenses  - 단위 백만원
# ---------------------------

response=requests.get(sales_loss_url)
response_main = json.loads(response.text)
#display(response_main)
#response_main

main_resp = get_json_response(sales_loss_url,"sales_loss")
main_df = pd.DataFrame(main_resp['StatisticSearch']['row'])

for i in range(2010,2014):
  sales_loss_url= "https://ecos.bok.or.kr/api/StatisticSearch/17RCR3XU541NV6BJAJJ7/json/kr/1/1000/501Y002/A/"+str(i)+"/2025"
  main_resp = get_json_response(sales_loss_url,"sales_loss")
  main_df = pd.DataFrame(main_resp['StatisticSearch']['row'])
  temp1_df=main_df[main_df['ITEM_NAME3']=='대손상각비']
  temp2_df=main_df[main_df['ITEM_NAME3']=='투자·유형자산 처분손실']
  temp3_df=main_df[main_df['ITEM_NAME3']=='기타영업외비용']
  if i==2010:
    writeoff_df=temp1_df
    loss_tan_ass_df=temp2_df
    non_op_exp_df=temp3_df
  else:
    writeoff_df=pd.concat([writeoff_df,temp1_df])
    loss_tan_ass_df=pd.concat([loss_tan_ass_df,temp2_df])
    non_op_exp_df=pd.concat([non_op_exp_df,temp3_df])


writeoff_df['대손상각비(단위:백만원)'] = pd.to_numeric(writeoff_df['DATA_VALUE'], errors='coerce')
loss_tan_ass_df['투자_유형자산_처분손실(단위:백만원)']= pd.to_numeric(loss_tan_ass_df['DATA_VALUE'])
non_op_exp_df['기타영업외비용(단위:백만원)']=pd.to_numeric(non_op_exp_df['DATA_VALUE'])

writeoff_df.reset_index(inplace=True)
loss_tan_ass_df.reset_index(inplace=True)
non_op_exp_df.reset_index(inplace=True)

incr_cred_loss_df = pd.concat([writeoff_df['대손상각비(단위:백만원)'],loss_tan_ass_df['투자_유형자산_처분손실(단위:백만원)'],non_op_exp_df['기타영업외비용(단위:백만원)']], axis=1)

incr_cred_loss_df['기타영업외비용(단위:백만원)'].fillna(non_op_exp_df['기타영업외비용(단위:백만원)'].min(), inplace=True)
incr_cred_loss_df['투자_유형자산_처분손실(단위:백만원)'].fillna(loss_tan_ass_df['투자_유형자산_처분손실(단위:백만원)'].min(), inplace=True)

incr_cred_loss_df['발생신용손실(단위:백만원)']=incr_cred_loss_df['대손상각비(단위:백만원)']+incr_cred_loss_df['투자_유형자산_처분손실(단위:백만원)']+incr_cred_loss_df['기타영업외비용(단위:백만원)']
incr_cred_loss_df



/tmp/ipython-input-3015319906.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  incr_cred_loss_df['기타영업외비용(단위:백만원)'].fillna(non_op_exp_df['기타영업외비용(단위:백만원)'].min(), inplace=True)
/tmp/ipython-input-3015319906.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when 

,대손상각비(단위:백만원),투자_유형자산_처분손실(단위:백만원),기타영업외비용(단위:백만원),발생신용손실(단위:백만원)
0,5108,10624.0,71298.0,87030.0
1,1524,162.0,3442.0,5128.0
2,3584,10461.0,67855.0,81900.0
3,2855,4441.0,20876.0,28172.0
4,2483,930.0,5161.0,8574.0
...,...,...,...,...
107,4249,64899.0,176708.0,245856.0
108,70683,162.0,3442.0,74287.0
109,2309,162.0,3442.0,5913.0
110,68374,162.0,3442.0,71978.0


## 기존 ECL 모델

In [ ]:
import requests
#xml 을 딕셔너리 구조로 보여주는 xmltodict 라이브러리 사용
import xmltodict
import pprint as pp
import requests
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import json

#ECOS_KEY = "17RCR3XU541NV6BJAJJ7"
pd_url = f"https://ecos.bok.or.kr/api/StatisticSearch/17RCR3XU541NV6BJAJJ7/json/kr/1/1000/722Y001/M/201001/201412"
ead_url = f"https://ecos.bok.or.kr/api/StatisticSearch/17RCR3XU541NV6BJAJJ7/json/kr/1/1000/801Y002/M/201001/201412"

def get_json_response(url, service_name="API"):
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"❌ {service_name} 요청 실패: {resp.status_code}")
        return None
    try:
        return resp.json()
    except ValueError:
        print(f"❌ {service_name} 응답이 JSON이 아님")
        return None

# ---------------------------
# 1. PD (부도확률, 722Y001)
# ---------------------------

response=requests.get(pd_url)
response_pd = json.loads(response.text)
#display(response_pd)
#response_pd

pd_resp = get_json_response(pd_url, "PD(ECOS)")
pd_df = pd.DataFrame(pd_resp['StatisticSearch']['row'])
pd_df['TIME'] = pd.to_datetime(pd_df['TIME'], format='%Y%m').dt.year
pd_df['부도확률(PD)'] = pd.to_numeric(pd_df['DATA_VALUE'], errors='coerce') / 100


# ---------------------------
# 2. EAD (부도_시_손실금액 -> 부도, 801Y002)
# ---------------------------

response=requests.get(ead_url)
response_ead = json.loads(response.text)
#display(response_ead)
#response_ead

ead_resp = get_json_response(ead_url, "EAD(ECOS)")
ead_df = pd.DataFrame(ead_resp['StatisticSearch']['row'])
ead_df['TIME'] = pd.to_datetime(ead_df['TIME'], format='%Y%m')
ead_df['부도_시_손실금액(EAD)(단위:십억원)'] = pd.to_numeric(ead_df['DATA_VALUE'], errors='coerce')
# ead_df['EAD'] = ead_df['EAD'].map(lambda x: f"{x:.1f}")


# ---------------------------
# 3. LGD (부도_시_손실확률)
# lgd_df['LGD']가 1000행이고, pd_df['PD'], ead_df['EAD']도 1000행이라고 가정
# ---------------------------

lgd_df=pd.read_csv("/content/LGD_mock_1000.csv")
ecl_df = pd.concat([
    pd_df['부도확률(PD)'].reset_index(drop=True),
    lgd_df['LGD'].reset_index(drop=True),
    ead_df['부도_시_손실금액(EAD)(단위:십억원)'].reset_index(drop=True)
], axis=1)

ecl_df.columns = ['부도확률(PD)', '부도_시_손실확률(LGD)', '부도_시_손실금액(EAD)(단위:십억원)']
ecl_df['기대신용손실(ECL)(단위:십억원)'] = ecl_df['부도확률(PD)'] * ecl_df['부도_시_손실확률(LGD)'] * ecl_df['부도_시_손실금액(EAD)(단위:십억원)']

ecl_df.dropna(inplace=True)

#pd_df['TIME'] = pd.to_datetime(pd_df['TIME'])  # 혹시 아직 datetime 형식이 아니라면 변환
#ecl_df['TIME'] = ecl_df['TIME'].dt.year

ecl_df['TIME']=pd_df['TIME']
ecl_df


,부도확률(PD),부도_시_손실확률(LGD),부도_시_손실금액(EAD)(단위:십억원),기대신용손실(ECL)(단위:십억원),TIME
0,0.0200,0.3617,44808.0,324.141072,2010
1,0.0241,0.8287,438377.0,8755.120780,2010
2,0.0300,0.5194,9783.0,152.438706,2010
3,0.0100,0.8890,2240.0,19.913600,2010
4,0.0125,0.3312,21918.8,90.743832,2010
...,...,...,...,...,...
386,0.0050,0.6145,330675.2,1015.999552,2014
387,0.0100,0.4653,15721.0,73.149813,2014
388,0.0050,0.5103,1107.0,2.824511,2014
389,0.0100,0.8506,17404.0,148.038424,2014


## 거시경제지표 (경제성장률, 실업률, 금리) 모음

In [ ]:
"""
기본공식:
조정된 ECL = 기본ECL × [1 + Σ(탄력성계수 × 변수변화율)]

시나리오: GDP -2%, 실업률 +1%p, 금리 +0.5%p

조정ECL = 100억 × [1 + (-0.3×(-2%) + 0.5×1% + 0.2×0.5%)]
       = 100억 × [1 + (0.6% + 0.5% + 0.1%)]
       = 100억 × 1.012
       = 101.2억원
"""

growth_url = f"https://ecos.bok.or.kr/api/StatisticSearch/17RCR3XU541NV6BJAJJ7/json/kr/1/1000/251Y003/A/2010/2014"
unempl_url = f"https://ecos.bok.or.kr/api/StatisticSearch/17RCR3XU541NV6BJAJJ7/json/kr/1/1000/901Y027/A/2010/2014"
interest_rate_url = f"https://ecos.bok.or.kr/api/StatisticSearch/17RCR3XU541NV6BJAJJ7/json/kr/1/1000/722Y001/A/2010/2014"

def get_json_response(url, service_name="API"):
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"❌ {service_name} 요청 실패: {resp.status_code}")
        return None
    try:
        return resp.json()
    except ValueError:
        print(f"❌ {service_name} 응답이 JSON이 아님")
        return None

# ---------------------------
# 1. 경제성장률 (251Y003)
# ---------------------------

response=requests.get(pd_url)
response_pd = json.loads(response.text)
#display(response_pd)

response_pd = get_json_response(growth_url)
growth_df = pd.DataFrame(response_pd['StatisticSearch']['row'])
#pd_df['TIME'] = pd.to_datetime(gdp_df['TIME'], format='%Y%m')
growth_df = growth_df[(growth_df['ITEM_NAME2']=="경제성장률") & (growth_df['ITEM_NAME1']=="한국")]
growth_df['경제성장률']=pd.to_numeric(growth_df['DATA_VALUE'], errors='coerce') / 100
growth_df.reset_index(inplace=True)

# ---------------------------
# 2. Unemployment 실업률 (901Y027)
# ---------------------------

response_pd = get_json_response(unempl_url)
unempl_df = pd.DataFrame(response_pd['StatisticSearch']['row'])
#pd_df['TIME'] = pd.to_datetime(gdp_df['TIME'], format='%Y%m')
unempl_df = unempl_df[unempl_df['ITEM_NAME1']=="실업률"]
unempl_df['실업률']=pd.to_numeric(unempl_df['DATA_VALUE'], errors="coerce")/100

unempl_df.dropna(subset=['실업률'], inplace=True)
unempl_df.reset_index(inplace=True)


# ---------------------------
# 3. Interest Rate (722Y001)
# ---------------------------

response_pd = get_json_response(interest_rate_url)
interest_rate_df = pd.DataFrame(response_pd['StatisticSearch']['row'])
#pd_df['TIME'] = pd.to_datetime(gdp_df['TIME'], format='%Y%m')
interest_rate_df = interest_rate_df[interest_rate_df['ITEM_NAME1']=="한국은행 기준금리"]
interest_rate_df['기준금리']=interest_rate_df['DATA_VALUE']
interest_rate_df.reset_index(inplace=True)

macroeco_df=pd.concat([growth_df['경제성장률'],unempl_df['실업률'],interest_rate_df['기준금리'], interest_rate_df['TIME']],axis=1)
macroeco_df

,경제성장률,실업률,기준금리,TIME
0,0.070,0.037,2.5,2010
1,0.037,0.034,3.25,2011
2,0.025,0.032,2.75,2012
3,0.033,0.031,2.5,2013
4,0.032,0.035,2,2014


## 해당 년도의 경제지표를 반영한 ECL 모델

In [ ]:
# TIME을 연도로 맞추기
ecl_df['YEAR'] = ecl_df['TIME']
macroeco_df['YEAR'] = pd.to_datetime(macroeco_df['TIME']).dt.year

# 두 DF merge
updated_ecl_df = pd.merge(ecl_df, macroeco_df, on="YEAR", how="left")

# 문자열 → 숫자 변환
updated_ecl_df['경제성장률'] = pd.to_numeric(updated_ecl_df['경제성장률'], errors='coerce')
updated_ecl_df['실업률'] = pd.to_numeric(updated_ecl_df['실업률'], errors='coerce')
updated_ecl_df['기준금리'] = pd.to_numeric(updated_ecl_df['기준금리'], errors='coerce')


"""
PD_조정계수 = 1 + w₁×GDP변화율 + w₂×실업률변화율 + w₃×금리변화율

w₁ = -0.4  (GDP↓ → PD↑, 가장 강한 영향)
w₂ = +0.4  (실업률↑ → PD↑, 가장 강한 영향)
w₃ = +0.2  (금리↑ → PD↑, 중간 영향)


LGD_조정계수 = 1 + w₄×GDP변화율 + w₅×실업률변화율 + w₆×금리변화율

w₄ = -0.2  (GDP↓ → 자산가치↓ → LGD↑)
w₅ = +0.1  (실업률↑ → 매각시장 악화 → LGD↑, 약한 영향)
w₆ = +0.3  (금리↑ → 할인율↑ → 회수가치↓ → LGD↑, 강한 영향)


EAD_조정계수 = 1 + w₇×GDP변화율 + w₈×실업률변화율 + w₉×금리변화율

w₇ = -0.1  (GDP↓ → 추가 자금 필요 → EAD↑, 약한 영향)
w₈ = +0.2  (실업률↑ → 생활비 충당 → EAD↑)
w₉ = +0.3  (금리↑ → 기존 저금리 최대 활용 → EAD↑, 강한 영향)


Updated ECL 최종 공식
조정된_PD = 기본_PD × PD_조정계수
조정된_LGD = 기본_LGD × LGD_조정계수
조정된_EAD = 기본_EAD × EAD_조정계수

ECL = 조정된_PD × 조정된_LGD × 조정된_EAD
"""

updated_ecl_df['경제지표_반영된_부도확률(PD)'] = updated_ecl_df['부도확률(PD)'] * (1 + updated_ecl_df['경제성장률'] * 0.4 + updated_ecl_df['실업률'] * 0.4 + updated_ecl_df['기준금리'] * 0.2)
updated_ecl_df['경제지표_반영된_부도_시_손실확률(LGD)']= updated_ecl_df['부도_시_손실확률(LGD)'] * (1 + updated_ecl_df['경제성장률'] * -0.2 + updated_ecl_df['실업률'] * 0.1 + updated_ecl_df['기준금리'] * 0.3)
updated_ecl_df['경제지표_반영된_부도_시_손실금액(EAD)(단위:십억원)']=  updated_ecl_df['부도_시_손실금액(EAD)(단위:십억원)'] *  (1 + updated_ecl_df['경제성장률'] * -0.1 + updated_ecl_df['실업률'] * 0.2 + updated_ecl_df['기준금리'] * 0.3)

# Updated ECL 계산
updated_ecl_df['경제지표_반영된_기대신용손실(Updated_ECL)(단위:십억원)'] = updated_ecl_df['경제지표_반영된_부도확률(PD)'] * updated_ecl_df['경제지표_반영된_부도_시_손실확률(LGD)'] * updated_ecl_df['경제지표_반영된_부도_시_손실금액(EAD)(단위:십억원)']

# 결과 확인

updated_ecl_df

,부도확률(PD),부도_시_손실확률(LGD),부도_시_손실금액(EAD)(단위:십억원),기대신용손실(ECL)(단위:십억원),TIME_x,YEAR,경제성장률,실업률,기준금리,TIME_y,경제지표_반영된_부도확률(PD),경제지표_반영된_부도_시_손실확률(LGD),경제지표_반영된_부도_시_손실금액(EAD)(단위:십억원),경제지표_반영된_기대신용손실(Updated_ECL)(단위:십억원)
0,0.0200,0.3617,44808.0,324.141072,2010,2010,0.070,0.037,2.5,2010,0.030856,0.629249,78431.92320,1522.843810
1,0.0241,0.8287,438377.0,8755.120780,2010,2010,0.070,0.037,2.5,2010,0.037181,1.441689,767335.10080,41132.342176
2,0.0300,0.5194,9783.0,152.438706,2010,2010,0.070,0.037,2.5,2010,0.046284,0.903600,17124.16320,716.170704
3,0.0100,0.8890,2240.0,19.913600,2010,2010,0.070,0.037,2.5,2010,0.015428,1.546593,3920.89600,93.555878
4,0.0125,0.3312,21918.8,90.743832,2010,2010,0.070,0.037,2.5,2010,0.019285,0.576189,38366.66752,426.322656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,0.0050,0.6145,330675.2,1015.999552,2014,2014,0.032,0.035,2.0,2014,0.007134,0.981418,530336.88576,3713.119581
387,0.0100,0.4653,15721.0,73.149813,2014,2014,0.032,0.035,2.0,2014,0.014268,0.743131,25213.33980,267.336735
388,0.0050,0.5103,1107.0,2.824511,2014,2014,0.032,0.035,2.0,2014,0.007134,0.815000,1775.40660,10.322588
389,0.0100,0.8506,17404.0,148.038424,2014,2014,0.032,0.035,2.0,2014,0.014268,1.358493,27912.53520,541.028163


In [ ]:
incr_cred_loss_df.to_csv('발생신용손실.csv', index=False, encoding='utf-8-sig')
ecl_df.to_csv('기대신용손실.csv', index=False, encoding='utf-8-sig')
macroeco_df.to_csv('거시경제지표.csv', index=False, encoding='utf-8-sig')
updated_ecl_df.to_csv('반영된_기대신용손실.csv', index=False, encoding='utf-8-sig')